# PREDICTION CODE FOR DEEP LEARNING PROJECT-I (RIME 2020)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv 
import pandas as pd
import math 
from tqdm import tqdm

## IMAGE LOADING & PRE PROCESSING 

In [2]:
image= cv.imread('BA_35991.jpg',0) #please load your test image here with file name.extension 

In [3]:
image= cv.resize(image,(32,32),interpolation = cv.INTER_AREA)

In [4]:
image=np.reshape(image, (1,32,32)) #changing size of image

In [5]:
# load numpy array from csv file
from numpy import load

## LOADING PARAMETERS

In [7]:
f1 = load('f1.npy',allow_pickle=True)

In [8]:
f1 = np.reshape(f1,(8,1,5,5)) 
f1.shape 

(8, 1, 5, 5)

In [9]:
f2 = load('f2.npy',allow_pickle=True)

In [10]:
f2 = np.reshape(f2,(5,8,5,5)) 
f2.shape

(5, 8, 5, 5)

In [11]:
w3 = load('w3.npy',allow_pickle=True)

In [12]:
w3 = np.reshape(w3,(128,720)) 
w3.shape

(128, 720)

In [13]:
w4 = load('w4.npy',allow_pickle=True)

In [14]:
w4 = np.reshape(w4,(8,128)) 
w4.shape

(8, 128)

In [15]:
b1 = load('b1.npy',allow_pickle=True)

In [16]:
b1 = np.reshape(b1,(8,1)) 
b1.shape

(8, 1)

In [17]:
b2 = load('b2.npy',allow_pickle=True)

In [18]:
b2 = np.reshape(b2,(5,1)) 
b2.shape

(5, 1)

In [19]:
b3 = load('b3.npy',allow_pickle=True)

In [20]:
b3 = np.reshape(b3,(128,1)) 
b3.shape

(128, 1)

In [21]:
b4 = load('b4.npy',allow_pickle=True)

In [22]:
b4 = np.reshape(b4,(8,1)) 
b4.shape

(8, 1)

In [23]:
parameters = [f1, f2, w3, w4, b1, b2, b3, b4]  #saving all parameters in a single list 

### DEFINING CONVOLUTION 

In [24]:
def convolution(image, filt, bias, s=1):
    '''
    Confolves `filt` over `image` using stride `s`
    '''
    (n_f, n_c_f, f, _) = filt.shape # filter dimensions
    n_c, in_dim, _ = image.shape # image dimensions
    
    out_dim = int((in_dim - f)/s)+1 # calculate output dimensions
    
    # ensure that the filter dimensions match the dimensions of the input image
    assert n_c == n_c_f, "Dimensions of filter must match dimensions of input image"
    
    out = np.zeros((n_f,out_dim,out_dim)) # create the matrix to hold the values of the convolution operation
    
    # convolve each filter over the image
    for curr_f in range(n_f):
        curr_y = out_y = 0
        # move filter vertically across the image
        while curr_y + f <= in_dim:
            curr_x = out_x = 0
            # move filter horizontally across the image 
            while curr_x + f <= in_dim:
                # perform the convolution operation and add the bias
                out[curr_f, out_y, out_x] = np.sum(filt[curr_f] * image[:,curr_y:curr_y+f, curr_x:curr_x+f]) + bias[curr_f]
                curr_x += s
                out_x += 1
            curr_y += s
            out_y += 1
        
    return out

### DEFINING MAXPOOLING 

In [25]:
def maxpool(image, f=2, s=2):
    
    #Downsample input `image` using a kernel size of `f` and a stride of `s`
    
    n_c, h_prev, w_prev = image.shape
    
    # calculate output dimensions after the maxpooling operation.
    h = int((h_prev - f)/s)+1 
    w = int((w_prev - f)/s)+1
    
    # create a matrix to hold the values of the maxpooling operation.
    downsampled = np.zeros((n_c, h, w)) 
    
    # slide the window over every part of the image using stride s. Take the maximum value at each step.
    for i in range(n_c):
        curr_y = out_y = 0
        # slide the max pooling window vertically across the image
        while curr_y + f <= h_prev:
            curr_x = out_x = 0
            # slide the max pooling window horizontally across the image
            while curr_x + f <= w_prev:
                # choose the maximum value within the window at each step and store it to the output matrix
                downsampled[i, out_y, out_x] = np.max(image[i, curr_y:curr_y+f, curr_x:curr_x+f])
                curr_x += s
                out_x += 1
            curr_y += s
            out_y += 1
            
    return downsampled

In [26]:
def softmax(raw_preds):
    
    out = np.exp(raw_preds) # exponentiate vector of raw predictions
    return out/np.sum(out) # divide the exponentiated vector by its sum. All values in the output sum to 1.

In [27]:
def categoricalCrossEntropy(probs, label):
    
    return -np.sum(label * np.log(probs)) # Multiply the desired output label by the log of the prediction, then sum all values in the vector


### DEFINING TEST FUNCTION 

In [28]:
def test(image, label, params, conv_s, pool_f, pool_s):
    
    [f1, f2, w3, w4, b1, b2, b3, b4] = params 
    
    # conv1()-relu-pooling-conv2()-relu-pooling-fc-ouput-softmax
    
    
    conv1 = convolution(image, f1, b1, conv_s) # first convolution operation
    conv1[conv1<=0] = 0 # pass through ReLU non-linearity
    
    pooled1 = maxpool(conv1, pool_f, pool_s) # first maxpooling operation
    
    conv2 = convolution(conv1, f2, b2, conv_s) # second convolution operation
    conv2[conv2<=0] = 0 # pass through ReLU non-linearity
    
    pooled = maxpool(conv2, pool_f, pool_s) # second maxpooling operation
    
    (nf2, dim2, _) = pooled.shape
    fc = pooled.reshape((nf2 * dim2 * dim2, 1)) # flatten pooled layer
    
    z = w3.dot(fc) + b3 # first dense layer
    z[z<=0] = 0 # pass through ReLU non-linearity
    
    out = w4.dot(z) + b4 # second dense layer
     
    probs = softmax(out) # predict class probabilities with the softmax activation function
    
        
    loss = categoricalCrossEntropy(probs, label) # categorical cross-entropy loss

        
        
    return loss,probs

## TESTING 

In [29]:
loss,prediction = test(image, label=8, params=parameters, conv_s=1, pool_f=2, pool_s=2)

## RESULTS  

In [30]:
prediction

array([[2.91973526e-03],
       [3.39239737e-05],
       [1.48719552e-06],
       [8.87065082e-05],
       [8.40039552e-06],
       [1.54029989e-02],
       [9.81146642e-01],
       [3.98105521e-04]])

In [31]:
max(prediction)

array([0.98114664])

In [32]:
index_min = np.argmax(prediction)
print("The predicted class is : ",index_min)

The predicted class is :  6


In [33]:
if index_min == 0:
    print ("The predicted class is Basophill ")

if index_min == 1:
    print ("The predicted class is Eosinophill ") 

if index_min == 2:
    print ("The predicted class is Erythroblast ") 
    
if index_min == 3:
    print ("The predicted class is Ig ") 
    
if index_min == 4:
    print ("The predicted class is Lymphocyte ")

if index_min == 5:
    print ("The predicted class is Monocyte ") 

if index_min == 6:
    print ("The predicted class is Neutrophill ") 
    
if index_min == 7:
    print ("The predicted class is Platelets ") 


The predicted class is Neutrophill 


In [34]:
loss

500.67758623475714